### Viscoelastic contact with consrained conjugate gradient method

Based on the discussion on 29/03/2024, we 

### Reference:

[1] Bugnicourt, R., P. Sainsot, N. Lesaffre, and A.A. Lubrecht. ‘Transient Frictionless Contact of a Rough Rigid Surface on a Viscoelastic Half-Space’. Tribology International 113 (September 2017): 279–85. https://doi.org/10.1016/j.triboint.2017.01.032.

[2] Van Dokkum, Jan Steven, and Lucia Nicola. ‘Green’s Function Molecular Dynamics Including Viscoelasticity’. Modelling and Simulation in Materials Science and Engineering 27, no. 7 (1 October 2019): 075006. https://doi.org/10.1088/1361-651X/ab3031.